In [1]:
# Webscrape EA / ED data

In [2]:
import pandas as pd
import requests

In [3]:
# Define function that can work around pd.read_html which
# undesirably (for this use case) removes html code.
def get_with_urls(location, start_code='<a href="', 
                  stop_code='" rel="noopener" target="_blank">', 
                  header=0, parser='---'):
    rget = requests.get(location)
    file = rget.text
    # Solution is to remove first portion of the <a href= tag.
    file = file.replace(start_code,'')
    # Solution also remove last portion of the <a href= tags.
    file = file.replace(stop_code,parser)
    tbls = pd.read_html(file, header=header)
    # Return data with a parser that will be useful to further parse.
    return(tbls)

# Get base url from the text. This is uesful as a uniqe identifier.
def get_base_url(longer):
    if (longer.find('www.') > -1) & (longer.find('.edu') > -1):
        start = longer.find('www.') + 4
        stop = longer.find('.edu') + 4
        base_url = longer[start:stop]
        return(base_url)
    elif longer.find('.edu') > -1:
        rev_start = longer[::-1].find('ude.')
        rev_stop = longer[::-1][rev_start:].find('.', 4)
        base_url = longer[::-1][rev_start:][:rev_stop][::-1]
        base_url = base_url.replace('ttp://','')
        base_url = base_url.replace('ttps://','')
        return(base_url)
    else:
        return('Base URL Not Detected')

def parse_name(pre_parsed, parser='---'):
    start = pre_parsed.find('---') + 3
    return(pre_parsed[start:])

In [4]:
# Get and process Early Decision data.
ED_tbls = get_with_urls('https://blog.prepscholar.com/early-decision-deadlines')
ED_df = ED_tbls[1]
ED_df['insturl'] = ED_df['School'].apply(get_base_url)
ED_df['ParsedName'] = ED_df['School'].apply(parse_name)
del ED_df['School']

In [5]:
# Get and process Early Decision data.
EA_tbls = get_with_urls('https://blog.prepscholar.com/early-action-deadlines')
EA_df = EA_tbls[1]
EA_df['insturl'] = EA_df['School'].apply(get_base_url)
EA_df['ParsedName'] = EA_df['School'].apply(parse_name)
del EA_df['School']

In [6]:
ED_df.head()

,State,Early Decision Deadline,Early Decision II Deadline,Early Decision Notification,insturl,ParsedName
0,NY,November 1,NaN,December 15,acphs.edu,Albany College of Pharmacy and Health Sciences
1,NY,December 1,NaN,December 15,alfred.edu,Alfred University
2,PA,November 15,February 1,Not reported,allegheny.edu,Allegheny College
3,DC,November 15,January 15,"December 31, February 15",american.edu,American University
4,MA,November 1,NaN,December 15,amherst.edu,Amherst College


In [7]:
EA_df.head()

,State,EA Deadline,Decision Notification,insturl,ParsedName
0,TX,November 1,—,acu.edu,Abilene Christian University
1,NY,December 1,December 31,adelphi.edu,Adelphi University
2,GA,November 15 / January 15,December 15 / February 15,agnesscott.edu,Agnes Scott College
3,MA,November 1 / December 15,December 15 / January 31,assumption.edu,Assumption College
4,AL,November 1,—,auburn.edu,Auburn University
